## Let's determine how to compute the velocities from a frame to another

In [95]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.coordinates import Galactic, ICRS, LSR
import numpy as np

In [97]:
gala = Galactic(l=121.17*u.degree, b=-21.57*u.degree,
            pm_l_cosb=4.8*u.mas/u.yr, pm_b=-15.16*u.mas/u.yr,
            radial_velocity=23.42*u.km/u.s)

print(gala.transform_to(ICRS()))

<ICRS Coordinate: (ra, dec) in deg
    (10.67919546, 41.27191022)
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (5.34876813, -14.9751888, 23.42)>


### Initialisation of the different velocity components 

In [180]:
U      = 10.6 
V      = 247
W      = 7.6

## Radial velocity

We have different components adding up for that total radial velocity. Let's derive it step by step, starting in the plane containing the disc of the MW. The following graph shows the different contributions of the different velocities.  We are going to denote these velocities with the name disc in order to know that these are just the ones in the plane of the disc. We can note that only W is here having a value of zero.

#### In the plane of the disc

In [203]:
rad_disc   = V*np.cos(31.17*np.pi/180) - U*np.cos(58.83*np.pi/180)

trans_disc = -V*np.sin(31.17*np.pi/180) - U*np.sin(58.83*np.pi/180) #pm_l

#### In the plane passing by the north galactic pole and the LOS

In [214]:
rad_pole_LOS   = rad_disc*np.cos(21.57*np.pi/180) - W*np.cos(68.43*np.pi/180) #final radial velocity

trans_pole_LOS = -rad_disc*np.sin(21.57*np.pi/180) - W*np.sin(68.43*np.pi/180) #pm_b

#### From there we would then like to know what are these velocities in km/s instead of mas/yr.

In [215]:
dist_l = trans_disc*365.25*86400
dist_b = trans_pole_LOS*365.25*86400

pm_l_final = dist_l/(780*3.08E16*4.84814E-12*np.cos(21.57))
pm_b_final = dist_b/(780*3.08E16*4.84814E-12)

In [219]:
gala = Galactic(l=121.17*u.degree, b=-21.57*u.degree,
            pm_l_cosb=-pm_l_final*u.mas/u.yr, pm_b=-pm_b_final*u.mas/u.yr,
            radial_velocity=-rad_pole_LOS*u.km/u.s)

Answer = gala.transform_to(ICRS())

v_ra  = (Answer.pm_ra_cosdec.value)/np.cos(41.27191022*np.pi/180)
v_dec = Answer.pm_dec.value

print(v_ra)
print(v_dec)

-55.13246520851354
20.925514474738435


In [217]:
RadDist = 780*3.08E16 #1kpc is 3.086E16km
YrToSec = 365.25*86400 #Years to seconds

v = RadDist*(v_ra*4.84814E-12)/YrToSec
print("The transverse velocity of such uncertainties would be about::", v)

v = RadDist*(v_dec*4.84814E-12)/YrToSec
print("The transverse velocity of such uncertainties would be about::", v)

The transverse velocity of such uncertainties would be about:: -203.48102501017482
The transverse velocity of such uncertainties would be about:: 77.23117618777395


In [233]:
RadDist = 780*3.08E16 #1kpc is 3.086E16km
YrToSec = 365.25*86400 #Years to seconds

v = RadDist*(37.6*4.84814E-12)/YrToSec
print("The transverse velocity of such uncertainties would be about::", v)

v = RadDist*(20.9*4.84814E-12)/YrToSec
print("The transverse velocity of such uncertainties would be about::", v)

The transverse velocity of such uncertainties would be about:: 138.77279950110275
The transverse velocity of such uncertainties would be about:: 77.13700823332573


In [213]:
def ObsVel(l_obj, b_obj): #Observed velocity
    #___ Description_____________________________________________________________________________________
    #Arguments:: l and b coordinates in degrees
    #Returns:: ...
    #____________________________________________________________________________________________________

    #Let's first define the velocity of the Sun in the MW. To do so we usually sum the circular velocity
    #of the Sun with a peculiar velocity, the latter having three components, U (pointing towards the ga-
    #lactic center), V (depends on convention, here in the direction of the rotation of the Sun) and W 
    #(pointing towards the North galactic pole). Here we use the values from Reid and al (2019)
    
    U      = 10.6  
    V      = 247
    W      = 7.6
    
    #We first work in 2D, looking at the MW from the north pole, and hence only considering U and V, so
    #for now we only have to deal with the l angle. In the following rad_disc is the component of the 
    #radial velocity in the plane of the MW and l_vel_disc the l component of the velocity. They are both
    #expressed in km/s
    
    if l_obj >= 0 and l_obj < 90:
        rad_disc   =   U*np.cos(l_obj*np.pi/180) + V*np.cos((90 - l_obj)*np.pi/180)
        l_vel_disc = - U*np.sin(l_obj*np.pi/180) + V*np.sin((90 - l_obj)*np.pi/180)
        
    if l_obj > 90 and l_obj < 180:
        rad_disc   = - U*np.cos((180 - l_obj)*np.pi/180) + V*np.cos((l_obj - 90)*np.pi/180)
        l_vel_disc = - U*np.cos((180 - l_obj)*np.pi/180) - V*np.cos((l_obj - 90)*np.pi/180)
        
    if l_obj > 180 and l_obj < 270:
        rad_disc   = - U*np.cos(l_obj*np.pi/180) - V*np.cos((90 - l_obj)*np.pi/180)
        l_vel_disc =   U*np.sin(l_obj*np.pi/180) - V*np.sin((90 - l_obj)*np.pi/180)
        
    if l_obj > 270 and l_obj < 360:
        rad_disc   =   U*np.cos((180 - l_obj)*np.pi/180) - V*np.cos((l_obj - 90)*np.pi/180)
        l_vel_disc =   U*np.cos((180 - l_obj)*np.pi/180) + V*np.cos((l_obj - 90)*np.pi/180)
        
    #Now we focus on the plane defined by (rad_disc, W)  
    if b_obj >= 0 and b_obj < 90:

SyntaxError: unexpected EOF while parsing (<ipython-input-213-f6353a3c1e46>, line 38)

In [221]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.coordinates as coord
import astropy.units as u

from astropy.visualization import astropy_mpl_style

plt.style.use(astropy_mpl_style)

In [232]:
c1 = coord.SkyCoord(ra=89.014303*u.degree, dec=13.924912*u.degree,
                    distance=(37.59*u.mas).to(u.pc, u.parallax()),
                    pm_ra_cosdec=pm_l_final*u.mas/u.yr,
                    pm_dec=-pm_b_final*u.mas/u.yr,
                    radial_velocity=-203*u.km/u.s,
                    frame='icrs')

gc1 = c1.transform_to(coord.Galactocentric)
print(gc1.v_x, gc1.v_y, gc1.v_z)

208.32981820797795 km / s 294.8422574609254 km / s 32.793136479748945 km / s


In [226]:
v_sun = [11.1, 244, 7.25] * (u.km / u.s)  # [vx, vy, vz]
gc_frame = coord.Galactocentric(
    galcen_distance=8*u.kpc,
    galcen_v_sun=v_sun,
    z_sun=0*u.pc)

gc2 = c1.transform_to(gc_frame)
print(gc2.v_x, gc2.v_y, gc2.v_z)

28.42795836058219 km / s 169.69916086163028 km / s 17.708316524432327 km / s
